In [41]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'sst2_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.sst2', 'data_parameters': {'name': 'sst2', 'train_batch_size': 64, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 10, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.00086}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.5}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'mps', 'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 10325, 'n': 12

In [42]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [43]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  53237


In [44]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...


In [45]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 1053/1053 [00:05<00:00, 195.69it/s, epoch=0/10, loss=0.297, lr=0.00086, metric_score=0.81, time=5.38] 


Epoch: 0, Test Loss: 0.47645197595868793, Test Score: 0.7786697247706422, Time Cost: 0.034262895584106445


100%|██████████| 1053/1053 [00:05<00:00, 184.46it/s, epoch=1/10, loss=0.179, lr=0.00043, metric_score=0.952, time=11.1]


Epoch: 1, Test Loss: 0.45901251477854593, Test Score: 0.8061926605504587, Time Cost: 0.034803152084350586


...

Epoch: 7, Test Loss: 0.48859440854617525, Test Score: 0.8038990825688074, Time Cost: 0.03393888473510742


100%|██████████| 1053/1053 [00:05<00:00, 182.71it/s, epoch=8/10, loss=0.117, lr=3.36e-6, metric_score=0.952, time=50.9] 


Epoch: 8, Test Loss: 0.4911715877907617, Test Score: 0.805045871559633, Time Cost: 0.03410029411315918


100%|██████████| 1053/1053 [00:05<00:00, 191.17it/s, epoch=9/10, loss=0.402, lr=1.68e-6, metric_score=0.857, time=56.5] 


Epoch: 9, Test Loss: 0.491443287049021, Test Score: 0.805045871559633, Time Cost: 0.03386497497558594
model checkpoint saving to ./ckpt/sst2_configs_checkpoint...


In [46]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [47]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
accuracy 0.805045871559633


In [48]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
